
# LoRA 그림체 학습 코랩 (모바일 최적화)

이 노트북은 Colab에서 LoRA를 활용한 그림체 학습을 진행하기 위한 전체 과정입니다.  
모바일에서도 순서대로 실행만 하면 됩니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir = "/content/drive/MyDrive/train_data"
output_dir = "/content/drive/MyDrive/output_lora"

In [ ]:
!pip install transformers==4.31.0 diffusers==0.21.4 huggingface_hub==0.16.4 --force-reinstall --no-deps
!pip install accelerate==0.21.0 peft==0.5.0

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")

prompt = "a portrait in your trained style"
image = pipe(prompt).images[0]
image.save(f"{output_dir}/test_sample.png")

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import CLIPTextModel, CLIPTokenizer

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

peft_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(text_encoder, peft_config)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    num_train_epochs=10,
    logging_dir=f"{output_dir}/logs",
    save_strategy="epoch"
)

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class LoraImageDataset(Dataset):
    def __init__(self, folder_path, prompt="a portrait in your trained style", image_size=512):
        self.image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.prompt = prompt
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        return {
            "pixel_values": self.transform(image),
            "input_ids": tokenizer(self.prompt, return_tensors="pt", padding="max_length", truncation=True).input_ids.squeeze(0)
        }

train_dataset = LoraImageDataset(train_dir)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()